In [473]:
from pymongo import MongoClient
import pandas as pd
import time
import re
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
from dotenv import load_dotenv
import os
import requests
import warnings
warnings.filterwarnings("ignore")
from pymongo import GEOSPHERE
import json
#from cartoframes.viz import Map, Layer, popup_element
import geopandas as gpd
import geopy.distance


In [474]:
nyork_df = pd.read_csv("./dataset/new_york_data.csv")


In [475]:
load_dotenv()

True

In [476]:
token_fsq = os.getenv("token")

Now we have a data set with locations in NY, we are going to take the top 3 based on the density, and show them on a map with ametnities around.  
    we're also choosing the top 3 locations that are next to each other, because we want our company to be around other companies that had raised more thn 10M$.  
    
( so far we have been trying to choose a location of a company who had raised 10M\$, so we need to check the surroundings as well)



In [477]:
map2 = Map(location = ["40.751892", "-73.987592"], zoom_start = 15)
#Optimum Energy
folium.Circle([40.751892, -73.987592],radius=1000).add_to(map2)
#Crispy Gamer
folium.Circle([40.750597, -73.985273],radius=1000).add_to(map2)
#Heavy
folium.Circle([40.753385, -73.989620],radius=1000).add_to(map2)

In [478]:
map2

In [479]:
#add crispy game location

In [480]:
icon = Icon(color = "blue",
            opacity = 0.1,
            prefix = "fa",
            icon = "building",
            icon_color = "black"
)
Crispy_Gamer = Marker(
    location = [40.750597, -73.985273],
    tooltip = "Crispy Gamer",
    icon=icon,
    radius = 1000
)
Crispy_Gamer.add_to(map2)

In [481]:
#add "heavy" location

In [482]:
icon = Icon(color = "blue",
            opacity = 0.1,
            prefix = "fa",
            icon = "building",
            icon_color = "black"
)
Heavy = Marker(
    location = [40.753385, -73.989620],
    tooltip = "Heavy",
    icon=icon,
    radius = 1000
)
Heavy.add_to(map2)

In [483]:
#add Optimum_Energy location

In [484]:
icon = Icon(color = "blue",
            opacity = 0.1,
            prefix = "fa",
            icon = "building",
            icon_color = "black",
)
Optimum_Energy = Marker(
    location = [40.751892, -73.987592],
    tooltip = "Optimum Energy",
    icon=icon
)
Optimum_Energy.add_to(map2)

In [485]:
def getNearbyStarbucks(lat, long, radius=1000):

    # create the API request URL
    url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{long}&radius={radius}&chains=ab4c54c0-d68a-012e-5619-003048cad9da&exclude_all_chains=false&sort=DISTANCE&limit=10"        
    # make the GET request
    headers = {
    "accept": "application/json",
    "Authorization": token_fsq
    }
    
    response = requests.get(url,headers=headers).json()    
    return response
     

In [486]:
response_star = getNearbyStarbucks("40.751892","-73.987592")

In [487]:
starbucks = []
for i in response_star["results"]:
    
    name = i["name"]
    latitude = i["geocodes"]["main"]["latitude"]
    longitude = i["geocodes"]["main"]["longitude"]
    distance = i["distance"]
    
    starbucks.append({"name":name, "latitude":latitude, "longitude":longitude, "distance_Optimum Energy": distance})
    
    starbucks

{'location': [40.749124, -73.991375], 'tooltip': 'Starbucks'}

In [536]:
df_cafe = pd.DataFrame(starbucks)


In [512]:
for index, row in df_cafe.iterrows():
    
    #1. MARKER without icon
    starbucks = {"location": [row["latitude"], row["longitude"]], "tooltip": row["name"]}
    
    #2. Icon
          
    icon = Icon (
            color="lightred",
            opacity = 0.6,
            prefix = "fa",
            icon="coffee",
            icon_color = "black"
        )
     #3. Marker
    new_marker = Marker(**starbucks, icon = icon, radius = 1000)
    
    #4. Add the Marker
    new_marker.add_to(map2)

In [513]:
def getNearbyLocation(lat, long, categorie, radius=1000):

    # create the API request URL
    url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{long}&radius={radius}&categories={categorie}&exclude_all_chains=false&sort=DISTANCE&limit=10"

    # make the GET request
    headers = {
    "accept": "application/json",
    "Authorization": token_fsq
    }    
    response = requests.get(url,headers=headers).json()    
    
    return response

In [514]:
night_life_response = getNearbyLocation("40.751892","-73.987592","13003")

In [515]:
night_life = []
for i in night_life_response["results"]:
    
    name = i["name"]
    latitude = i["geocodes"]["main"]["latitude"]
    longitude = i["geocodes"]["main"]["longitude"]
    distance = i["distance"]
    
    night_life.append({"name":name, "latitude":latitude, "longitude":longitude, "distance_Optimum Energy": distance})
    
    night_life

In [538]:
df_night_life = pd.DataFrame(night_life)


In [517]:
for index, row in df_night_life.iterrows():
    
    #1. MARKER without icon
    nightlife = {"location": [row["latitude"], row["longitude"]], "tooltip": row["name"]}
    
    #2. Icon
          
    icon = Icon (
            color="purple",
            opacity = 0.6,
            prefix = "fa",
            icon="glass",
            icon_color = "black"
        )
     #3. Marker
    new_marker1 = Marker(**nightlife, icon = icon, radius = 1000)
    
    #4. Add the Marker
    new_marker1.add_to(map2)

In [518]:
#transportation
transportation_response = getNearbyLocation("40.751892","-73.987592","19047,19049,19046")#19047 rail
#19049 taxi
#19046 metro


coords_1 = (40.751892,-73.987592)
coords_2 = (40.752206,-73.989137)
print (geopy.distance.geodesic(coords_1, coords_2).km)

In [519]:
transportation = []
for i in transportation_response["results"]:
    
    name = i["name"]
    latitude = i["geocodes"]["main"]["latitude"]
    longitude = i["geocodes"]["main"]["longitude"]
    distance = i["distance"]
    
    transportation.append({"name":name, "latitude":latitude, "longitude":longitude, "distance_Optimum Energy": distance})
    
    transportation



In [520]:
df_transportation = pd.DataFrame(transportation)


In [521]:
for index, row in df_transportation.iterrows():
    
    #1. MARKER without icon
    transportation = {"location": [row["latitude"], row["longitude"]], "tooltip": row["name"]}
    
    #2. Icon
          
    icon = Icon (
            color="yellow",
            opacity = 0.6,
            prefix = "fa",
            icon="car",
            icon_color = "white"
        )
     #3. Marker
    new_marker2 = Marker(**transportation, icon = icon, radius = 1000)
    
    #4. Add the Marker
    new_marker2.add_to(map2)

In [522]:
preschool_response = getNearbyLocation("40.751892","-73.987592","12056")

In [523]:
preschool = []
for i in preschool_response["results"]:
    
    name = i["name"]
    latitude = i["geocodes"]["main"]["latitude"]
    longitude = i["geocodes"]["main"]["longitude"]
    distance = i["distance"]
    
    preschool.append({"name":name, "latitude":latitude, "longitude":longitude, "distance_Optimum Energy": distance})
    
    preschool

In [524]:
df_preschool = pd.DataFrame(preschool)

In [525]:
for index, row in df_preschool.iterrows():
    
    #1. MARKER without icon
    preschool = {"location": [row["latitude"], row["longitude"]], "tooltip": row["name"]}
    
    #2. Icon
          
    icon = Icon (
            color="pink",
            opacity = 0.6,
            prefix = "fa",
            icon="child",
            icon_color = "black"
        )
     #3. Marker
    new_marker3 = Marker(**preschool, icon = icon, radius = 1000)
    
    #4. Add the Marker
    new_marker3.add_to(map2)

In [526]:
vegan_response = getNearbyLocation("40.751892","-73.987592","13377")


In [527]:
vegan_rest = []
for i in vegan_response["results"]:
    
    name = i["name"]
    latitude = i["geocodes"]["main"]["latitude"]
    longitude = i["geocodes"]["main"]["longitude"]
    distance = i["distance"]
    
    vegan_rest.append({"name":name, "latitude":latitude, "longitude":longitude, "distance_Optimum Energy": distance})
    
    vegan_rest

In [528]:
df_vegan_rest = pd.DataFrame(vegan_rest)


In [529]:
for index, row in df_vegan_rest.iterrows():
    
    #1. MARKER without icon
    vegan_rest = {"location": [row["latitude"], row["longitude"]], "tooltip": row["name"]}
    
    #2. Icon
          
    icon = Icon (
            color="lightgreen",
            opacity = 0.6,
            prefix = "fa",
            icon="leaf",
            icon_color = "black"
        )
     #3. Marker
    new_marker4 = Marker(**vegan_rest, icon = icon, radius = 1000)
    
    #4. Add the Marker
    new_marker4.add_to(map2)

In [530]:
pet_response = getNearbyLocation("40.751892","-73.987592","11134")


In [531]:
pet_service = []
for i in pet_response["results"]:
    
    name = i["name"]
    latitude = i["geocodes"]["main"]["latitude"]
    longitude = i["geocodes"]["main"]["longitude"]
    distance = i["distance"]
    
    pet_service.append({"name":name, "latitude":latitude, "longitude":longitude, "distance_Optimum Energy": distance})
    
    pet_service

In [532]:
df_pet_service = pd.DataFrame(pet_service)


In [533]:
for index, row in df_pet_service.iterrows():
    
    #1. MARKER without icon
    pet_service = {"location": [row["latitude"], row["longitude"]], "tooltip": row["name"]}
    
    #2. Icon
          
    icon = Icon (
            color="lightblue",
            opacity = 0.6,
            prefix = "fa",
            icon="paw",
            icon_color = "white"
        )
     #3. Marker
    new_marker5 = Marker(**pet_service, icon = icon, radius = 1000)
    
    #4. Add the Marker
    new_marker5.add_to(map2)

In [534]:
map2

In [422]:
#up to 50